In [ ]:
!pip3 install segmentation_models_pytorch
!pip3 install warmup_scheduler
#!pip uninstall torch -y
#!pip install torch==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64

In [ ]:
#!/usr/bin/env python
# coding: utf-8
# with help from Jiahao
# created by Yu Yang
# Date Apr 2021
# update Date Oct 2021
# update Date Jan 2023

# updated with mixed supervision
# updated with 2023 MICCAI sub
# image abd mask for FA data
# image and label for WA data

# In[1]: import module


import torch.utils.data as Data
import torch
import segmentation_models_pytorch as smp
import segmentation_models_pytorch.losses as smp_losses
import segmentation_models_pytorch.utils as smp_utils
from torchsummary import summary
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import cv2
import datetime
import random
import sys
#from torchsummary import summary
#from tensorboardX import SummaryWriter
from torch.optim.lr_scheduler import StepLR
from warmup_scheduler import GradualWarmupScheduler
from copy import deepcopy
import torchvision.utils as vutils
import ssl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# In[2]: define GPU


#get_ipython().run_line_magic('env', 'CUDA_DEVICE_ORDER=PCI_BUS_ID')
#get_ipython().run_line_magic('env', 'CUDA_VISIBLE_DEVICES=0')
torch.set_num_threads(6)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# In[2]: define seed (global, just for guidance, have to re-call them everytime when it's needed

seed_cus = 1

random.seed(seed_cus)
np.random.seed(seed_cus)
torch.manual_seed(seed_cus)
torch.cuda.manual_seed(seed_cus)
torch.cuda.manual_seed_all(seed_cus)

#torch.set_deterministic(True)
torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
#os.environ['PYTHONHASHSEED'] = str(2)

# may want to check/print the train/val data first

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(seed_cus)



In [ ]:

# In[4]: define dataloader


import torch.utils.data as Data
class My_Datasets(Data.Dataset):
    def __init__(self, img_dir, mask_dir, transform1=None,transform2=None):
        super().__init__()
        self.img_dir = img_dir
        self.mask_dir = mask_dir


        self.img_list = os.listdir(self.img_dir)
        self.mask_list = os.listdir(self.mask_dir)

        self.temp = []

        for img_name in os.listdir(self.img_dir):
          mask_name = img_name.split('.')[0] + "_0000.nii.png"
          img = cv2.imread(os.path.join(self.img_dir,img_name)) # here we load as RBG
          mask= cv2.imread(os.path.join(self.mask_dir,mask_name)) # here we load as RBG

          if type(mask) == type(None) or type(img) == type(None):
            self.temp += [img_name]

        print(self.temp)
        for name in self.temp:
          mask_name = name.split('.')[0] + "_0000.nii.png"
          self.img_list.remove(name)
          self.mask_list.remove(mask_name)



        self.transform1 = transform1
        self.transform2 = transform2

    def get_blacklisted(self):
      return self.temp

    def __getitem__(self, index):
        img_name = self.img_list[index]
        mask_name = self.img_list[index].split('.')[0] + "_0000.nii.png"


        if img_name.endswith('.png'):
            img = cv2.imread(os.path.join(self.img_dir,self.img_list[index])) # here we load as RBG
            mask= cv2.imread(os.path.join(self.mask_dir,mask_name)) # here we load as RBG
            #print(mask_name)

            if type(mask) == type(None) or type(img) == type(None):
              print(i)
              print(mask_name)

            else:
              LABImg = img/[255.0]
              LABmask = mask/[255.0]


              LABImg = cv2.resize(LABImg,(224,224))
              LABmask = cv2.resize(LABmask,(224,224))


              LABImg = LABImg.astype(np.float32)
              LABmask = LABmask.astype(np.float32)
              LABmask = LABmask[:,:,0]



              if self.transform1:
                LABImg = self.transform1(LABImg)
                LABmask = self.transform2(LABmask)


              return LABImg, LABmask, img_name

    def __len__(self):
        return len(self.img_list)


# In[5]: define transform


transforms_img = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
])
transforms_mask = transforms.Compose([
    transforms.ToTensor(),
])


In [ ]:
# In[6]: define data path and load data


img_path = '/content/drive/My Drive/Processed data/LGE'
mask_path = '/content/drive/My Drive/Processed data/masks'


batch_size_train = 8
batch_size_val = 8

train_set = My_Datasets(img_path,mask_path, transform1=transforms_img,transform2=transforms_mask)
n_train = len(train_set)
split = n_train // 5

[]


In [ ]:

names = os.listdir(img_path)

#group names by same patient
grouped_names = {}

blacklisted = train_set.get_blacklisted()

for i in range(len(names)):
  base = names[i].split('_')[0]

  if names[i] not in blacklisted:
    if len(grouped_names) == 0:
      grouped_names[base] = [names[i]]
    else:
      if base in grouped_names:
        grouped_names[base] += [names[i]]
      else:
        grouped_names[base] = [names[i]]

indices = list(range(n_train))

In [ ]:
#Attach an index to each img
for i in range(len(train_set)):
  tri = train_set[i]
  base = tri[2].split('_')[0]
  for x in range(len(grouped_names[base])):
    name = grouped_names[base][x]
    if name == tri[2]:
      grouped_names[base][x] = [name, i]

In [ ]:
#Get indices for training, validation, and testing. Put images in correct folders
length = len(train_set)
test_len = length // 5
val_len = length // 5

train_indices = []
val_indices = []
test_indices = []

test_names = []

train_len = length - test_len - val_len
for patient in grouped_names:
  while len(train_indices) < train_len:
    for index in range(len(grouped_names[patient])):
      train_indices += [grouped_names[patient][index][1]]
      if type(grouped_names[patient][index][1]) == str:
        print(grouped_names[patient][index])
    break

  else:
    while len(val_indices) < val_len:
      for index in range(len(grouped_names[patient])):
        val_indices += [grouped_names[patient][index][1]]
      break
    else:
      while len(test_indices) < test_len:
        for index in range(len(grouped_names[patient])):
          test_indices += [grouped_names[patient][index][1]]
          test_names += [grouped_names[patient][index][0]]
        break

In [ ]:
#check to make sure no indices are repeated
check = set()
for name in train_indices:
  check.add(name)

for name in val_indices:
  check.add(name)

for name in test_indices:
  check.add(name)

if len(check) == len(train_set):
  print("All good! There are no duplicates")
else:
  print("There are duplicates")

All good! There are no duplicates


In [ ]:
LGE_source = '/content/drive/My Drive/Processed data/LGE'
LGE_destination = '/content/drive/My Drive/Internship_24_Ryan/Heart/Data/test/LGE/'

mask_source = '/content/drive/My Drive/Processed data/masks/'
mask_destination = '/content/drive/My Drive/Internship_24_Ryan/Heart/Data/test/masks/'

Cine_source = '/content/drive/My Drive/Processed data/Cine/'
Cine_destination = '/content/drive/My Drive/Internship_24_Ryan/Heart/Data/test/Cine/'


for f in test_names:
    if f not in blacklisted:
      src_path = os.path.join(LGE_source, f)
      dst_path = os.path.join(LGE_destination, f)
      os.rename(src_path, dst_path)

      mask_name = f.split('.')[0] + "_0000.nii.png"
      src_path = os.path.join(mask_source, mask_name)
      dst_path = os.path.join(mask_destination, mask_name)
      os.rename(src_path, dst_path)

      Cine_name = f.split('.')[0] + "_0000.nii.tiff"
      src_path = os.path.join(Cine_source, Cine_name)
      dst_path = os.path.join(Cine_destination, Cine_name)
      os.rename(src_path, dst_path)
